In [2]:
import ee
import geemap
from cantor import *
ee.Initialize()
gez = ee.FeatureCollection("projects/sig-misc-ee/assets/sbp/ecofloristic_zones")
CONTINENTAL_REGIONS = ee.FeatureCollection("projects/sig-misc-ee/assets/sbp/continental_regions")


In [34]:
def carbon_growth_function(stand_age:ee.Image, paramters_stack:ee.Image):
  #// b0(1-EXP(-b2Age))^b2
  carbonHa = ee.Image(0).expression("(b0 *( 1 - exp)) ** b2",{
    'b0': paramters_stack.select('b0'),
    'exp': (stand_age.multiply(paramters_stack.select('b1').multiply(-1))).exp(),
    'b2': ee.Image(paramters_stack.select('b2'))
  }).rename('carbon');
  
  return carbonHa


def get_areas(image, geometry, scale=100):
    
    image = image.rename("image").round()
    pixelArea = ee.Image.pixelArea().divide(10000)
    reducer = ee.Reducer.sum().group(1, "image")

    areas = (
        pixelArea.addBands(image)
        .reduceRegion(reducer=reducer, geometry=geometry, scale=100, maxPixels=1e12)
        .get("groups")
    )

    areas_list = ee.List(areas).map(lambda i: ee.Dictionary(i).get("sum"))

    total = areas_list.reduce(ee.Reducer.sum())

    sum_img = image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=100,
        maxPixels=1e13,
    )

    return areas, total


In [3]:
# gez.aggregate_array('GEZ_TERM').distinct().getInfo()

humid_gez = [
    'Tropical rainforest',
    'Tropical moist deciduous forest'
    'Temperate oceanic forest',
    'Temperate continental forest',
    'Boreal coniferous forest',
    'Subtropical humid forest',
]
dry_gez = [
    'Temperate steppe',
    'Subtropical desert',
    'Subtropical steppe',
    'Tropical shrubland',
    'Tropical dry forest',
    'Tropical desert',
    'Subtropical dry forest',
    'Temperate desert',
    'Boreal tundra woodland',
]

excluded = [ 
    'Water',
    'No data',
    'Polar',
]
unknown = [
    'Boreal mountain system',
    'Tropical mountain system',
    'Subtropical mountain system',
    'Temperate mountain system',
]

In [4]:
gez_map = ee.Dictionary({
#     1 humid
    'Tropical rainforest':1,
    'Tropical moist deciduous forest':1,
    'Temperate oceanic forest':1,
    'Temperate continental forest':1,
    'Boreal coniferous forest':1,
    'Subtropical humid forest':1,
#     2dry
    'Temperate steppe':2,
    'Subtropical desert':2,
    'Subtropical steppe':2,
    'Tropical shrubland':2,
    'Tropical dry forest':2,
    'Tropical desert':2,
    'Subtropical dry forest':2,
    'Temperate desert':2,
    'Boreal tundra woodland':2,
#     3/4 nd
    'Water':3,
    'No data':3,
    'Polar':3,
    'Boreal mountain system':4,
    'Tropical mountain system':4,
    'Subtropical mountain system':4,
    'Temperate mountain system':4,
})


In [5]:

continental_map = ee.Dictionary({
    'Asia':5,
    'Asia (insular)':5,
    'Indian Ocean':5,
    'Australia':5,
    'New Zealand':5,
    
    'Africa':4,
    'Europe':3,
    'South America':2,
    'North America':1,
    

    
    'Pacific Ocean':6,
    'Atlantic Ocean':6,
    'Antarctica':6,
    'Arctic Ocean':6,
})


In [42]:
gez_region_code = gez.map(lambda f: f.set('climate_code', gez_map.get(f.get('GEZ_TERM'))))
continential_region_code = CONTINENTAL_REGIONS.map(lambda f: f.set('continential_code', continental_map.get(f.get('REGION'))))
print(continential_region_code.first().getInfo()['properties'])
empt = ee.Image()
gez_region_img = empt.paint(gez_region_code,'climate_code')
continential_region_img = empt.paint(continential_region_code,'continential_code')
paired = eePair(gez_region_img, continential_region_img).int()

{'REGION': 'Asia', 'continential_code': 5}


In [43]:
Map = geemap.Map()
Map.addLayer(CONTINENTAL_REGIONS)
Map.addLayer(gez_region_img)
Map.addLayer(continential_region_img)
Map.addLayer(paired)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [44]:
aoi = ee.FeatureCollection(Map.draw_last_feature)
samples = paired.int().stratifiedSample(numPoints=1, region=aoi, scale=1000)
print(samples.size().getInfo())


                                  

24


In [10]:
uvalues = samples.aggregate_array('constant').distinct().getInfo()
print(list(map(depair, uvalues)))
print(uvalues)
# test = paired.remap([4, 7, 8, 11, 12, 13, 16, 17, 18, 19, 23, 24, 25, 26, 31, 32, 33, 34, 40, 41, 42, 50, 51, 61],
#                       [1, 1, 1, 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])
# Map.addLayer(test)

[(1.0, 1.0), (2.0, 1.0), (1.0, 2.0), (3.0, 1.0), (2.0, 2.0), (1.0, 3.0), (4.0, 1.0), (3.0, 2.0), (2.0, 3.0), (1.0, 4.0), (4.0, 2.0), (3.0, 3.0), (2.0, 4.0), (1.0, 5.0), (4.0, 3.0), (3.0, 4.0), (2.0, 5.0), (1.0, 6.0), (4.0, 4.0), (3.0, 5.0), (2.0, 6.0), (4.0, 5.0), (3.0, 6.0), (4.0, 6.0)]
[4, 7, 8, 11, 12, 13, 16, 17, 18, 19, 23, 24, 25, 26, 31, 32, 33, 34, 40, 41, 42, 50, 51, 61]


In [11]:
con_cs = {
    '5':'asia',
    '4':'africa',
    '3':'europe',
    '2':'south amrc',
    '1':'north amrc',
    '6':'na'
}
gez_cs = {
    '1':'humid',
    '2':'dry'
    # '3':'na',
    
}
dp = list(map(depair, uvalues))
# manually update "code" field in carbon.json based on values. 
dp = [{'gez':gez_cs.get(str(int(i[0])),'na'), 'con':con_cs.get(str(int(i[1])),'na')} for i in dp]
zipd = zip(uvalues,dp)
list(zipd)

[(4, {'gez': 'humid', 'con': 'north amrc'}),
 (7, {'gez': 'dry', 'con': 'north amrc'}),
 (8, {'gez': 'humid', 'con': 'south amrc'}),
 (11, {'gez': 'na', 'con': 'north amrc'}),
 (12, {'gez': 'dry', 'con': 'south amrc'}),
 (13, {'gez': 'humid', 'con': 'europe'}),
 (16, {'gez': 'na', 'con': 'north amrc'}),
 (17, {'gez': 'na', 'con': 'south amrc'}),
 (18, {'gez': 'dry', 'con': 'europe'}),
 (19, {'gez': 'humid', 'con': 'africa'}),
 (23, {'gez': 'na', 'con': 'south amrc'}),
 (24, {'gez': 'na', 'con': 'europe'}),
 (25, {'gez': 'dry', 'con': 'africa'}),
 (26, {'gez': 'humid', 'con': 'asia'}),
 (31, {'gez': 'na', 'con': 'europe'}),
 (32, {'gez': 'na', 'con': 'africa'}),
 (33, {'gez': 'dry', 'con': 'asia'}),
 (34, {'gez': 'humid', 'con': 'na'}),
 (40, {'gez': 'na', 'con': 'africa'}),
 (41, {'gez': 'na', 'con': 'asia'}),
 (42, {'gez': 'dry', 'con': 'na'}),
 (50, {'gez': 'na', 'con': 'asia'}),
 (51, {'gez': 'na', 'con': 'na'}),
 (61, {'gez': 'na', 'con': 'na'})]

In [46]:
import json
# inputs, json path, paired img
path = r'C:\Users\johnj\Documents\SIG\19.restoration-fao\2022_seplan\notebooks\carbon.json'
# paired
with open(path) as json_file:
    data = json.load(json_file)
data

# make images of coefs
# update base img where code
# area * value
out = ee.Image(0)
for k, v in data.items():
    print(v)
    coe1, coe2, coe3 = v['parameters']
    coe_stack = ee.Image.cat([ee.Image(coe1), ee.Image(coe2), ee.Image(coe3)]).select([0,1,2],['b0','b1','b2'])
    v['carbon_40y'] = carbon_growth_function(ee.Image(40), coe_stack)
    t = carbon_growth_function_center(ee.Image(100), coe_stack)
    out = out.where(paired.eq(v['code']), v['carbon_40y'])


pixelArea = ee.Image.pixelArea().divide(10000)
Map.addLayer(out.selfMask(),{},'final maps?')
Map.addLayer(out.multiply(pixelArea),{},'AGB')

{'name': 'Natural Asia Dry', 'parameters': [6.952180603672807, 0.07708632118229979, 2.155374937783795], 'code': 33}
{'name': 'Natural Asia Humid', 'parameters': [22.20201288628687, 0.08282659433268577, 1.6575632681092896], 'code': 26}
{'name': 'Natural Euro Humid Dry', 'parameters': [6.673966194081617, 0.06740175651808789, 2.25457077327527], 'code': 18}
{'name': 'Natural Euro Humid Dry', 'parameters': [6.673966194081617, 0.06740175651808789, 2.25457077327527], 'code': 13, 'note': 'repeat since Europe curve is for both humid and dry.'}
{'name': 'Natural Afica Humid', 'parameters': [9.00912152868533, 0.06691874765644773, 2.2140421801368606], 'code': 19}
{'name': 'Central America Dry', 'parameters': [8.66429742100409, 0.2096291437364193, 1.7082408651411614], 'code': 7}
{'name': 'Central America Humid', 'parameters': [8.461890294323121, 0.056914894706377964, 2.17789147266434], 'code': 4}
{'name': 'South America Dry', 'parameters': [7.387321640157009, 0.09216975743131285, 2.1575324061267205

In [41]:
image = out.multiply(pixelArea)
geometry = ee.FeatureCollection(Map.draw_last_feature)
areas = (
    pixelArea.addBands(image)
    .reduceRegion(reducer=ee.Reducer.sum(), geometry=geometry, scale=100, maxPixels=1e12)
    # .get("groups")
)

areas, total = get_areas(out,  ee.FeatureCollection(Map.draw_last_feature))
print(areas.getInfo())
print(total.getInfo())

[{'image': 0, 'sum': 20656373.800121263}, {'image': 75, 'sum': 52594.46942112582}, {'image': 111, 'sum': 202751796.02882567}]
223460764.29836807
